# Train

In [1]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict

from ptb import PTB
from utils import to_var, idx2word, expierment_name, AttributeDict
from model_gumbel import SentenceVAE

In [2]:
# data_dir = 'data/simple-examples'
data_dir = 'data/eccos'
epoch=20
latent_size=16
gumbel_tau=0.1
kl_k=0.001
kl_x0=10000

args = {
    'data_dir': data_dir,
    'create_data': False,
    'max_sequence_length': 50,
    'min_occ': 1,
    'test': True,

    'epochs': epoch,
    'batch_size': 32,
    'learning_rate': 0.001,

    'embedding_size': 300,
    'rnn_type': 'gru',
    'hidden_size': 256,
    'num_layers': 1,
    'bidirectional': False,
    'latent_size': latent_size,
    'word_dropout': 0,
    'embedding_dropout': 0.5,

    'anneal_function': 'logistic',
    'k': kl_k,
    'x0': kl_x0,

    'print_every': 50,
    'tensorboard_logging': True,
    'logdir': '/root/user/work/logs/',
    'save_model_path': 'bin',
    'expierment_name': f'copy2copy_gumbel_latent{latent_size}_epoch{epoch}_tau={gumbel_tau}_k={kl_k}_x0={kl_x0}', 
    
    'gumbel_tau': gumbel_tau,
}

args = AttributeDict(args)

args.rnn_type = args.rnn_type.lower()
args.anneal_function = args.anneal_function.lower()

assert args.rnn_type in ['rnn', 'lstm', 'gru']
assert args.anneal_function in ['logistic', 'linear']
assert 0 <= args.word_dropout <= 1
args

<AttrDict{'data_dir': 'data/eccos', 'create_data': False, 'max_sequence_length': 50, 'min_occ': 1, 'test': True, 'epochs': 20, 'batch_size': 32, 'learning_rate': 0.001, 'embedding_size': 300, 'rnn_type': 'gru', 'hidden_size': 256, 'num_layers': 1, 'bidirectional': False, 'latent_size': 16, 'word_dropout': 0, 'embedding_dropout': 0.5, 'anneal_function': 'logistic', 'k': 0.001, 'x0': 10000, 'print_every': 50, 'tensorboard_logging': True, 'logdir': '/root/user/work/logs/', 'save_model_path': 'bin', 'expierment_name': 'copy2copy_gumbel_latent16_epoch20_tau=0.1_k=0.001_x0=10000', 'gumbel_tau': 0.1}>

## load data

In [3]:
%%time
import itertools
splits = ['train', 'valid'] + (['test'] if args.test else [])
datasets = OrderedDict()
print(f'loading {args.data_dir}')
for split, src_tgt in itertools.product(splits, ['src', 'tgt']):
    key = (split, src_tgt)
    print(key)
    datasets[key] = PTB(
        data_dir=f'{args.data_dir}/{src_tgt}',
        split=split,
        create_data=args.create_data,
        max_sequence_length=args.max_sequence_length if src_tgt == 'tgt' else args.max_sequence_length_src,
        min_occ=args.min_occ
    )
    print(f'vocab: {datasets[key].vocab_size}, records: {len(datasets[key].data)}')

loading data/eccos
('train', 'src')
vocab: 5619, records: 30726
('train', 'tgt')
vocab: 12106, records: 30726
('valid', 'src')
vocab: 5619, records: 7682
('valid', 'tgt')
vocab: 12106, records: 7682
('test', 'src')
vocab: 5619, records: 9603
('test', 'tgt')
vocab: 12106, records: 9603
CPU times: user 1.39 s, sys: 101 ms, total: 1.49 s
Wall time: 1.48 s


In [4]:
# 実際のデータ確認
def ids2text(id_list, ptb):
    return ' '.join([ptb.i2w[f'{i}'] for i in id_list])

_ptb_src = datasets[('train', 'src')]
_ptb_tgt = datasets[('train', 'tgt')]
index = str(101)
_sample_src, _sample_tgt = _ptb_src.data[index], _ptb_tgt[index]
print(f'■ src-input \n{ids2text(_sample_src["input"], _ptb_src)}')
print(f'■ src-target \n{ids2text(_sample_src["target"], _ptb_src)}')
print(f'■ tgt-input\n{ids2text(_sample_tgt["input"], _ptb_tgt)}')
print(f'■ tgt-target\n{ids2text(_sample_tgt["target"], _ptb_tgt)}')

■ src-input 
<sos> bbクリーム 進化 クリーム
■ src-target 
bbクリーム 進化 クリーム <eos>
■ tgt-input
<sos> bbクリーム の 進化 版 ? cc クリーム が 気 に なる ... ! <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
■ tgt-target
bbクリーム の 進化 版 ? cc クリーム が 気 に なる ... ! <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


## build model

In [5]:
from ptb import SOS_INDEX, EOS_INDEX, PAD_INDEX, UNK_INDEX

In [6]:
%load_ext autoreload

In [7]:
%autoreload
model = SentenceVAE(
    vocab_size=datasets[('train', 'tgt')].vocab_size,
    sos_idx=SOS_INDEX,
    eos_idx=EOS_INDEX,
    pad_idx=PAD_INDEX,
    unk_idx=UNK_INDEX,
    max_sequence_length=args.max_sequence_length,
    embedding_size=args.embedding_size,
    rnn_type=args.rnn_type,
    hidden_size=args.hidden_size,
    word_dropout=args.word_dropout,
    embedding_dropout=args.embedding_dropout,
    latent_size=args.latent_size,
    num_layers=args.num_layers,
    bidirectional=args.bidirectional,
    
    # bow loss
    # bow_hidden_size=256,
    use_bow_loss=False,
    
    is_gumbel=True,
    gumbel_tau=args.gumbel_tau,
)

if torch.cuda.is_available():
    model = model.cuda()

In [8]:
model

SentenceVAE(
  (embedding): Embedding(12106, 300)
  (decoder_embedding): Embedding(12106, 300)
  (embedding_dropout): Dropout(p=0.5, inplace=False)
  (encoder_rnn): GRU(300, 256, batch_first=True)
  (decoder_rnn): GRU(300, 256, batch_first=True)
  (hidden2gumbel): Linear(in_features=256, out_features=12106, bias=True)
  (hidden2mean): Linear(in_features=300, out_features=16, bias=True)
  (hidden2logv): Linear(in_features=300, out_features=16, bias=True)
  (latent2hidden): Linear(in_features=16, out_features=256, bias=True)
  (outputs2vocab): Linear(in_features=256, out_features=12106, bias=True)
)

## log

In [9]:
print(f'tensorboard logging: {args.tensorboard_logging}')
ts = time.strftime('%Y-%b-%d-%H:%M:%S', time.gmtime())
if args.tensorboard_logging:
    writer = SummaryWriter(os.path.join(args.logdir, expierment_name(args,ts)))
    writer.add_text("model", str(model))
    writer.add_text("args", str(args))
    writer.add_text("ts", ts)
    
save_model_path = os.path.join(args.save_model_path, ts)
os.makedirs(save_model_path)

tensorboard logging: True


## optimizer

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.Tensor
step = 0

In [11]:
datasets.keys()

odict_keys([('train', 'src'), ('train', 'tgt'), ('valid', 'src'), ('valid', 'tgt'), ('test', 'src'), ('test', 'tgt')])

In [12]:
ae_datasets = {split: dataset for (split, src_tgt), dataset in datasets.items() if src_tgt == 'tgt'}
print(ids2text(ae_datasets['train'][0]['input'], ae_datasets['train']))
print(ids2text(ae_datasets['train'][0]['target'], ae_datasets['train']))

<sos> 大人気 <unk> ピンク ♡ コンビニ 買える 「 さくら リップ 」 に 限定 色 が 登場 <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
大人気 <unk> ピンク ♡ コンビニ 買える 「 さくら リップ 」 に 限定 色 が 登場 <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [13]:
_datasets = {}
for split in splits:
    src_dataset = datasets[(split, 'src')]
    tgt_dataset = datasets[(split, 'tgt')]
    assert len(src_dataset) == len(tgt_dataset)
    dataset = []
    for i in range(len(src_dataset)):
        src_set, tgt_set = src_dataset[i], tgt_dataset[i]
        _data = {}
        _data.update({f'src_{k}': v for k,v in tgt_set.items()})
        _data.update({f'tgt_{k}': v for k,v in tgt_set.items()})
        dataset.append(_data)
    _datasets[split] = dataset

In [14]:
_datasets.keys()

dict_keys(['train', 'valid', 'test'])

In [15]:
_datasets['train'][0]

{'src_input': array([ 2,  4,  1,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'src_target': array([ 4,  1,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  3,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'src_length': 16,
 'tgt_input': array([ 2,  4,  1,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'tgt_target': array([ 4,  1,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,  3,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 'tgt_length': 16}

In [16]:
train_target_ptb = datasets[('train', 'tgt')]
train_target_ptb

In [17]:
%pdb on
for epoch in range(args.epochs):

    for split in splits:

        data_loader = DataLoader(
            dataset=_datasets[split],
            batch_size=args.batch_size,
            shuffle=split=='train',
            num_workers=cpu_count(),
            pin_memory=torch.cuda.is_available()
        )

        tracker = defaultdict(tensor)

        # Enable/Disable Dropout
        if split == 'train':
            model.train()
        else:
            model.eval()

        for iteration, batch in enumerate(data_loader):
            
            batch_size = batch['src_input'].size(0)
            
            for k, v in batch.items():
                if torch.is_tensor(v):
                    batch[k] = to_var(v)
            
            # loss calculation | additional Condition ！!!
            cal_dict = model(batch['tgt_input'], batch['tgt_length'])
            logp, mean, logv, z = cal_dict['logp'], cal_dict['mean'], cal_dict['logv'], cal_dict['z']
            
            loss_dict = model.loss(logp, batch['tgt_target'], batch['tgt_length'], mean, logv, args.anneal_function, step, args.k, args.x0, bow_input=z)
            loss, NLL_loss, KL_weight, KL_loss, avg_bow_loss = loss_dict['loss'], loss_dict['NLL_loss'], loss_dict['KL_weight'], loss_dict['KL_loss'], loss_dict.get('avg_bow_loss')

            # backward + optimization
            if split == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                step += 1

            # bookkeepeing
            tracker['ELBO'] = torch.cat((tracker['ELBO'], loss.data.view(1)))

            if args.tensorboard_logging:
                writer.add_scalar("%s/ELBO"%split.upper(), loss.data.item(), epoch*len(data_loader) + iteration)
                writer.add_scalar("%s/NLL Loss"%split.upper(), NLL_loss.data.item()/batch_size, epoch*len(data_loader) + iteration)
                writer.add_scalar("%s/KL Loss"%split.upper(), KL_loss.data.item()/batch_size, epoch*len(data_loader) + iteration)
                writer.add_scalar("%s/KL Weight"%split.upper(), KL_weight, epoch*len(data_loader) + iteration)
                if avg_bow_loss is not None:
                    writer.add_scalar("%s/BOW Loss"%split.upper(), avg_bow_loss, epoch*len(data_loader) + iteration)

            if iteration % args.print_every == 0 or iteration+1 == len(data_loader):
                print_text = "%s Batch %04d/%i, Loss %9.4f, NLL-Loss %9.4f, KL-Loss %9.4f, KL-Weight %6.3f"%(split.upper(), iteration, len(data_loader)-1, loss.data.item(), NLL_loss.data.item()/batch_size, KL_loss.data.item()/batch_size, KL_weight)
                if avg_bow_loss is not None:
                    print_text += ', BOW Loss %9.4f,'%(avg_bow_loss)
                print(print_text)

            if split == 'valid':
                if 'target_sents' not in tracker:
                    tracker['target_sents'] = list()
                tracker['target_sents'] += idx2word(batch['tgt_target'].data, i2w=train_target_ptb.get_i2w(), pad_idx=PAD_INDEX)
                tracker['z'] = torch.cat((tracker['z'], z.data), dim=0)

        print("%s Epoch %02d/%i, Mean ELBO %9.4f"%(split.upper(), epoch, args.epochs, torch.mean(tracker['ELBO'])))

        if args.tensorboard_logging:
            writer.add_scalar("%s-Epoch/ELBO"%split.upper(), torch.mean(tracker['ELBO']), epoch)

        # save a dump of all sentences and the encoded latent space
        if split == 'valid':
            dump = {'target_sents':tracker['target_sents'], 'z':tracker['z'].tolist()}
            if not os.path.exists(os.path.join('dumps', ts)):
                os.makedirs('dumps/'+ts)
            with open(os.path.join('dumps/'+ts+'/valid_E%i.json'%epoch), 'w') as dump_file:
                json.dump(dump,dump_file)

        # save checkpoint
        if split == 'train':
            checkpoint_path = os.path.join(save_model_path, "E%i.pytorch"%(epoch))
            torch.save(model.state_dict(), checkpoint_path)
            print("Model saved at %s"%checkpoint_path)

Automatic pdb calling has been turned ON
TRAIN Batch 0000/960, Loss  180.5082, NLL-Loss  180.5081, KL-Loss    3.7285, KL-Weight  0.000
TRAIN Batch 0050/960, Loss  132.1216, NLL-Loss  132.1213, KL-Loss    4.6285, KL-Weight  0.000
TRAIN Batch 0100/960, Loss  118.5340, NLL-Loss  118.5338, KL-Loss    3.7039, KL-Weight  0.000
TRAIN Batch 0150/960, Loss  112.9919, NLL-Loss  112.9917, KL-Loss    3.9133, KL-Weight  0.000
TRAIN Batch 0200/960, Loss  109.7903, NLL-Loss  109.7901, KL-Loss    3.5411, KL-Weight  0.000
TRAIN Batch 0250/960, Loss  107.4168, NLL-Loss  107.4165, KL-Loss    4.0667, KL-Weight  0.000
TRAIN Batch 0300/960, Loss  109.7005, NLL-Loss  109.7003, KL-Loss    3.3464, KL-Weight  0.000
TRAIN Batch 0350/960, Loss  112.8913, NLL-Loss  112.8911, KL-Loss    3.5252, KL-Weight  0.000
TRAIN Batch 0400/960, Loss  110.8961, NLL-Loss  110.8958, KL-Loss    4.5946, KL-Weight  0.000
TRAIN Batch 0450/960, Loss  101.8585, NLL-Loss  101.8582, KL-Loss    4.7253, KL-Weight  0.000
TRAIN Batch 0500/96

In [18]:
def debug_memory():
    import collections, gc, resource, torch
    print('maxrss = {}'.format(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss))
    tensors = collections.Counter((str(o.device), o.dtype, tuple(o.shape)) for o in gc.get_objects() if torch.is_tensor(o))
    for line in tensors.items():
        print('{}\t{}'.format(*line))
debug_memory()

maxrss = 2082004


/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:101: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


('cpu', torch.int64, (3,))	1
('cuda:0', torch.int64, (3, 50))	4
('cuda:0', torch.int64, (3,))	2
('cuda:0', torch.float32, (3, 16))	3
('cuda:0', torch.float32, (3, 20, 12106))	1
('cuda:0', torch.float32, ())	3
('cuda:0', torch.float32, (301,))	1
('cuda:0', torch.float32, (12106, 300))	3
('cuda:0', torch.float32, (768, 300))	6
('cuda:0', torch.float32, (768, 256))	6
('cuda:0', torch.float32, (768,))	12
('cuda:0', torch.float32, (12106, 256))	6
('cuda:0', torch.float32, (12106,))	6
('cuda:0', torch.float32, (16, 300))	6
('cuda:0', torch.float32, (16,))	6
('cuda:0', torch.float32, (256, 16))	3
('cuda:0', torch.float32, (256,))	3
